<a href="https://colab.research.google.com/github/fabianoborgesbsb/APC_study/blob/main/APC_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import pandas_gbq
import matplotlib.pyplot as plt

In [24]:
import os

file_path = '/content/drive/My Drive/APC_study.ipynb'
print(os.path.exists(file_path))  # Deve retornar: True



True


In [25]:
import nbformat

with open(file_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

for cell in nb.cells:
    if 'metadata' in cell and 'widgets' in cell['metadata']:
        del cell['metadata']['widgets']

# Salvar como novo notebook (opcional)
with open('/content/drive/My Drive/APC_study_fixed.ipynb', 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)



# Average APC each country

In [26]:
%%bigquery --project=insyspo

SELECT countries.country_code,
  ANY_VALUE(countries.country) AS country,
  AVG(CAST(apcs.value_usd AS FLOAT64)) AS avg_value_usd
FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
JOIN (
  SELECT DISTINCT apcs.id, authorships.institution_id
  FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
  JOIN `insyspo.publicdb_openalex_2024_04_rm.works_authorships` AS authorships
  ON apcs.id = authorships.work_id
) AS institutions
ON apcs.id = institutions.id
JOIN `insyspo.publicdb_openalex_2024_04_rm.institutions` AS countries
ON institutions.institution_id = countries.id
GROUP BY countries.country_code
ORDER BY avg_value_usd DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,country_code,country,avg_value_usd
0,CK,Cook Islands,5045.000000
1,IM,Isle of Man,3726.630290
2,AX,Aland Islands,3666.062500
3,GB,United Kingdom,3622.919975
4,US,United States,3564.686030
...,...,...,...
222,NI,Nicaragua,1001.088838
223,TJ,Tajikistan,964.795914
224,SV,El Salvador,868.427632
225,ID,Indonesia,514.570688


# Average APC every year each country

In [ ]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS userdb_luis_fabiano_borges.apcs;
CREATE TABLE userdb_luis_fabiano_borges.apcs AS (
  SELECT countries.country_code,
    institutions.publication_year,
    ANY_VALUE(countries.country) AS country,
    AVG(CAST(apcs.value_usd AS FLOAT64)) AS avg_value_usd,
    COUNT(DISTINCT apcs.id) AS works
  FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
  JOIN (
    SELECT DISTINCT apcs.id, authorships.institution_id, works.publication_year
    FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works_authorships` AS authorships
    ON apcs.id = authorships.work_id
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works` AS works
    ON works.id = apcs.id
  ) AS institutions
  ON apcs.id = institutions.id
  JOIN `insyspo.publicdb_openalex_2024_04_rm.institutions` AS countries
  ON institutions.institution_id = countries.id
  WHERE institutions.publication_year >= 1990
    AND institutions.publication_year <= 2024
  GROUP BY countries.country_code, institutions.publication_year
  HAVING AVG(CAST(apcs.value_usd AS FLOAT64)) > 0.0
  ORDER BY institutions.publication_year, countries.country_code
)


ERROR:
 ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7e4a021a5ba0>)


In [ ]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs

Query is running:   0%|          |

Downloading:   0%|          |

# Each field

In [ ]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS userdb_luis_fabiano_borges.apcs_engineering;
CREATE TABLE userdb_luis_fabiano_borges.apcs_engineering AS (
  SELECT countries.country_code,
    institutions.publication_year,
    ANY_VALUE(countries.country) AS country,
    AVG(CAST(apcs.value_usd AS FLOAT64)) AS avg_value_usd,
    COUNT(DISTINCT apcs.id) AS works
  FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
  JOIN (
    SELECT DISTINCT apcs.id, authorships.institution_id,
      works.publication_year
    FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works_authorships` AS authorships
    ON apcs.id = authorships.work_id
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works` AS works
    ON works.id = apcs.id
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works_topics` AS works_topics
    ON works.id = works_topics.work_id
    JOIN `insyspo.publicdb_openalex_2024_04_rm.topics` AS topics
    ON works_topics.topic_id = topics.id
    WHERE topics.field = 22
  ) AS institutions
  ON apcs.id = institutions.id
  JOIN `insyspo.publicdb_openalex_2024_04_rm.institutions` AS countries
  ON institutions.institution_id = countries.id
  WHERE institutions.publication_year >= 1990
  AND institutions.publication_year <= 2024
  GROUP BY countries.country_code, institutions.publication_year
  HAVING AVG(CAST(apcs.value_usd AS FLOAT64)) > 0.0
  ORDER BY institutions.publication_year, countries.country_code
);

Query is running:   0%|          |

""


# Correct tables for missing country codes

Aggregate

In [ ]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df.columns

Index(['country_code', 'publication_year', 'country', 'avg_value_usd',
       'works'],
      dtype='object')

In [ ]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

In [ ]:
df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df[df['country']=='Netherlands']

,country_code,publication_year,country,avg_value_usd,works
4965,NL,1990,Netherlands,3652.344606,6692
4966,NL,1991,Netherlands,3650.152591,6913
4967,NL,1992,Netherlands,3664.445768,7692
4968,NL,1993,Netherlands,3708.915442,8264
4969,NL,1994,Netherlands,3678.380257,8918
4970,NL,1995,Netherlands,3716.301863,9537
4971,NL,1996,Netherlands,3722.898974,10130
4972,NL,1997,Netherlands,3716.978325,10815
4973,NL,1998,Netherlands,3743.393525,10937
4974,NL,1999,Netherlands,3820.963455,10789


In [ ]:
df.drop(row_drop,axis=0, inplace=True)

In [ ]:
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs',if_exists ='replace')

100%|██████████| 1/1 [00:00<00:00, 780.92it/s]


Computer Science

In [ ]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs_computer_science

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df.drop(row_drop,axis=0, inplace=True)
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_computer_science',if_exists ='replace')


100%|██████████| 1/1 [00:00<00:00, 1694.67it/s]


Medicine


In [ ]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs_medicine

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df.drop(row_drop,axis=0, inplace=True)
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_medicine',if_exists ='replace')


100%|██████████| 1/1 [00:00<00:00, 790.78it/s]


Engineering

In [ ]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs_engineering

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df.drop(row_drop,axis=0, inplace=True)
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_engineering',if_exists ='replace')


100%|██████████| 1/1 [00:00<00:00, 5599.87it/s]


Physics

In [ ]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs_physics

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df.drop(row_drop,axis=0, inplace=True)
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_physics',if_exists ='replace')


100%|██████████| 1/1 [00:00<00:00, 1467.57it/s]
